In [1]:
import pandas as pd

In [2]:
df = pd.read_excel(
    "/home/xujialiu/mnt-4T-xujialiu/my_lightning_model/data_cleansing/get_macula_table/macula_table.xlsx",
    index_col=0,
)

In [3]:
df

,label,macula_filename,pid,eye,date,time
0,1,STDR001_OD_20150728_105647_Color_001.tif,STDR001,OD,2015-07-28,10:56:47
1,2,STDR001_OD_20160714_101437_Color_001.tif,STDR001,OD,2016-07-14,10:14:37
2,2,STDR001_OD_20160714_101504_Color_001.tif,STDR001,OD,2016-07-14,10:15:04
3,2,STDR001_OD_20160714_101606_Color_001.tif,STDR001,OD,2016-07-14,10:16:06
4,2,STDR001_OD_20170117_110223_Color_001.tif,STDR001,OD,2017-01-17,11:02:23
...,...,...,...,...,...,...
11634,0,STDR696_OD_20230613_145815_Color_001.tif,STDR696,OD,2023-06-13,14:58:15
11635,0,STDR696_OS_20210629_160514_Color_001.tif,STDR696,OS,2021-06-29,16:05:14
11636,0,STDR696_OS_20211216_152742_Color_001.tif,STDR696,OS,2021-12-16,15:27:42
11637,1,STDR696_OS_20220614_160840_Color_001.tif,STDR696,OS,2022-06-14,16:08:40


In [4]:
grouped = df.groupby(["pid", "eye", "date"])["label"].nunique()

In [5]:
grouped

pid      eye  date      
STDR001  OD   2015-07-28    1
              2016-07-14    1
              2017-01-17    1
         OS   2015-07-28    1
              2016-07-14    1
                           ..
STDR696  OD   2023-06-13    1
         OS   2021-06-29    1
              2021-12-16    1
              2022-06-14    1
              2023-06-13    1
Name: label, Length: 8592, dtype: int64

In [6]:
grouped[grouped > 1]

pid      eye  date      
STDR039  OD   2022-01-14    2
STDR176  OD   2018-10-26    2
STDR433  OD   2019-03-29    2
STDR435  OS   2022-08-02    2
STDR454  OS   2019-07-22    2
STDR481  OD   2019-05-30    2
STDR490  OD   2020-03-06    2
STDR528  OS   2022-01-03    2
STDR611  OD   2017-08-31    2
STDR631  OD   2017-10-24    2
STDR641  OD   2017-11-23    2
Name: label, dtype: int64

In [7]:
df_selected_sorted = df[["pid", "eye", "date", "label"]].sort_values(
    ["pid", "eye", "date", "label"]
)

In [8]:
df_selected_sorted[
    (df_selected_sorted.pid == "STDR039")
    & (df_selected_sorted.date == "2022-01-14")
    & (df_selected_sorted.eye == "OD")
]  # 确认, 大的label在下面, 小的label在上面

,pid,eye,date,label
443,STDR039,OD,2022-01-14,0
442,STDR039,OD,2022-01-14,1


In [41]:
df_drop_duplicates = df_selected_sorted.drop_duplicates(
    subset=["pid", "eye", "date"], keep="last"
)

In [42]:
df_drop_duplicates[
    (df_selected_sorted.pid == "STDR039")
    & (df_selected_sorted.date == "2022-01-14")
    & (df_selected_sorted.eye == "OD")
]  # 确认保留了较大的label

/tmp/ipykernel_9037/1454753019.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_drop_duplicates[


,pid,eye,date,label
442,STDR039,OD,2022-01-14,1


In [108]:
df_drop_duplicates  # 这里的label其实是fix_inconsistent_label

,pid,eye,date,label
0,STDR001,OD,2015-07-28,1
3,STDR001,OD,2016-07-14,2
5,STDR001,OD,2017-01-17,2
8,STDR001,OS,2015-07-28,0
10,STDR001,OS,2016-07-14,0
...,...,...,...,...
11634,STDR696,OD,2023-06-13,0
11635,STDR696,OS,2021-06-29,0
11636,STDR696,OS,2021-12-16,0
11637,STDR696,OS,2022-06-14,1


In [59]:
df_shift_date_label = df_drop_duplicates.copy()

In [61]:
df_shift_date_label["date_up_shift_1"] = df_shift_date_label.groupby(["pid", "eye"])[
    "date"
].shift(-1)
df_shift_date_label["date_up_shift_2"] = df_shift_date_label.groupby(["pid", "eye"])[
    "date"
].shift(-2)
df_shift_date_label["date_up_shift_3"] = df_shift_date_label.groupby(["pid", "eye"])[
    "date"
].shift(-3)
df_shift_date_label["date_up_shift_4"] = df_shift_date_label.groupby(["pid", "eye"])[
    "date"
].shift(-4)

df_shift_date_label

,pid,eye,date,label,date_up_shift_1,date_up_shift_2,date_up_shift_3,date_up_shift_4
0,STDR001,OD,2015-07-28,1,2016-07-14,2017-01-17,NaN,NaN
3,STDR001,OD,2016-07-14,2,2017-01-17,NaN,NaN,NaN
5,STDR001,OD,2017-01-17,2,NaN,NaN,NaN,NaN
8,STDR001,OS,2015-07-28,0,2016-07-14,2017-01-17,NaN,NaN
10,STDR001,OS,2016-07-14,0,2017-01-17,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
11634,STDR696,OD,2023-06-13,0,NaN,NaN,NaN,NaN
11635,STDR696,OS,2021-06-29,0,2021-12-16,2022-06-14,2023-06-13,NaN
11636,STDR696,OS,2021-12-16,0,2022-06-14,2023-06-13,NaN,NaN
11637,STDR696,OS,2022-06-14,1,2023-06-13,NaN,NaN,NaN


In [67]:
df_shift_date_label["label_up_shift_1"] = df_shift_date_label.groupby(["pid", "eye"])[
    "label"
].shift(-1)
df_shift_date_label["label_up_shift_2"] = df_shift_date_label.groupby(["pid", "eye"])[
    "label"
].shift(-2)
df_shift_date_label["label_up_shift_3"] = df_shift_date_label.groupby(["pid", "eye"])[
    "label"
].shift(-3)
df_shift_date_label["label_up_shift_4"] = df_shift_date_label.groupby(["pid", "eye"])[
    "label"
].shift(-4)

df_shift_date_label

,pid,eye,date,label,date_up_shift_1,date_up_shift_2,date_up_shift_3,date_up_shift_4,date_diff_1,label_up_shift_1,label_up_shift_2,label_up_shift_3,label_up_shift_4
0,STDR001,OD,2015-07-28,1,2016-07-14,2017-01-17,NaN,NaN,352.0,2.0,2.0,NaN,NaN
3,STDR001,OD,2016-07-14,2,2017-01-17,NaN,NaN,NaN,187.0,2.0,NaN,NaN,NaN
5,STDR001,OD,2017-01-17,2,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,STDR001,OS,2015-07-28,0,2016-07-14,2017-01-17,NaN,NaN,352.0,0.0,0.0,NaN,NaN
10,STDR001,OS,2016-07-14,0,2017-01-17,NaN,NaN,NaN,187.0,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11634,STDR696,OD,2023-06-13,0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11635,STDR696,OS,2021-06-29,0,2021-12-16,2022-06-14,2023-06-13,NaN,170.0,0.0,1.0,1.0,NaN
11636,STDR696,OS,2021-12-16,0,2022-06-14,2023-06-13,NaN,NaN,180.0,1.0,1.0,NaN,NaN
11637,STDR696,OS,2022-06-14,1,2023-06-13,NaN,NaN,NaN,364.0,1.0,NaN,NaN,NaN


In [68]:
for i in range(4):
    df_shift_date_label[f"date_up_shift_{i+1}"] = pd.to_datetime(
        df_shift_date_label[f"date_up_shift_{i+1}"]
    )

df_shift_date_label["date"] = pd.to_datetime(df_shift_date_label["date"])

df_shift_date_label

,pid,eye,date,label,date_up_shift_1,date_up_shift_2,date_up_shift_3,date_up_shift_4,date_diff_1,label_up_shift_1,label_up_shift_2,label_up_shift_3,label_up_shift_4
0,STDR001,OD,2015-07-28,1,2016-07-14,2017-01-17,NaT,NaT,352.0,2.0,2.0,NaN,NaN
3,STDR001,OD,2016-07-14,2,2017-01-17,NaT,NaT,NaT,187.0,2.0,NaN,NaN,NaN
5,STDR001,OD,2017-01-17,2,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN
8,STDR001,OS,2015-07-28,0,2016-07-14,2017-01-17,NaT,NaT,352.0,0.0,0.0,NaN,NaN
10,STDR001,OS,2016-07-14,0,2017-01-17,NaT,NaT,NaT,187.0,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11634,STDR696,OD,2023-06-13,0,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN
11635,STDR696,OS,2021-06-29,0,2021-12-16,2022-06-14,2023-06-13,NaT,170.0,0.0,1.0,1.0,NaN
11636,STDR696,OS,2021-12-16,0,2022-06-14,2023-06-13,NaT,NaT,180.0,1.0,1.0,NaN,NaN
11637,STDR696,OS,2022-06-14,1,2023-06-13,NaT,NaT,NaT,364.0,1.0,NaN,NaN,NaN


In [69]:
for i in range(1, 5):
    df_shift_date_label[f"date_diff_{i}"] = (
        df_shift_date_label[f"date_up_shift_{i}"] - df_shift_date_label["date"]
    ).dt.days

df_shift_date_label

,pid,eye,date,label,date_up_shift_1,date_up_shift_2,date_up_shift_3,date_up_shift_4,date_diff_1,label_up_shift_1,label_up_shift_2,label_up_shift_3,label_up_shift_4,date_diff_2,date_diff_3,date_diff_4
0,STDR001,OD,2015-07-28,1,2016-07-14,2017-01-17,NaT,NaT,352.0,2.0,2.0,NaN,NaN,539.0,NaN,NaN
3,STDR001,OD,2016-07-14,2,2017-01-17,NaT,NaT,NaT,187.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
5,STDR001,OD,2017-01-17,2,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,STDR001,OS,2015-07-28,0,2016-07-14,2017-01-17,NaT,NaT,352.0,0.0,0.0,NaN,NaN,539.0,NaN,NaN
10,STDR001,OS,2016-07-14,0,2017-01-17,NaT,NaT,NaT,187.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11634,STDR696,OD,2023-06-13,0,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11635,STDR696,OS,2021-06-29,0,2021-12-16,2022-06-14,2023-06-13,NaT,170.0,0.0,1.0,1.0,NaN,350.0,714.0,NaN
11636,STDR696,OS,2021-12-16,0,2022-06-14,2023-06-13,NaT,NaT,180.0,1.0,1.0,NaN,NaN,544.0,NaN,NaN
11637,STDR696,OS,2022-06-14,1,2023-06-13,NaT,NaT,NaT,364.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [85]:
for i in range(1, 5):
    mask = (df_shift_date_label[f"date_diff_{i}"] >= (356 - 30)) & (
        df_shift_date_label[f"date_diff_{i}"] <= (356 + 30)
    )
    df_shift_date_label[f"label_365_{i}"] = df_shift_date_label[f"label_up_shift_{i}"][
        mask
    ]

df_shift_date_label["label_365"] = (
    df_shift_date_label[["label_365_1", "label_365_2", "label_365_3", "label_365_4"]]
    .bfill(axis=1)
    .iloc[:, 0]
)

In [86]:
for i in range(1, 5):
    mask = (df_shift_date_label[f"date_diff_{i}"] >= (183 - 30)) & (
        df_shift_date_label[f"date_diff_{i}"] <= (183 + 30)
    )
    df_shift_date_label[f"label_183_{i}"] = df_shift_date_label[f"label_up_shift_{i}"][
        mask
    ]


df_shift_date_label["label_183"] = (
    df_shift_date_label[["label_183_1", "label_183_2", "label_183_3", "label_183_4"]]
    .bfill(axis=1)
    .iloc[:, 0]
)
df_shift_date_label

,pid,eye,date,label,date_up_shift_1,date_up_shift_2,date_up_shift_3,date_up_shift_4,date_diff_1,label_up_shift_1,...,label_365,label_365_1,label_365_2,label_365_3,label_365_4,label_183_1,label_183_2,label_183_3,label_183_4,label_183
0,STDR001,OD,2015-07-28,1,2016-07-14,2017-01-17,NaT,NaT,352.0,2.0,...,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,STDR001,OD,2016-07-14,2,2017-01-17,NaT,NaT,NaT,187.0,2.0,...,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,2.0
5,STDR001,OD,2017-01-17,2,NaT,NaT,NaT,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,STDR001,OS,2015-07-28,0,2016-07-14,2017-01-17,NaT,NaT,352.0,0.0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,STDR001,OS,2016-07-14,0,2017-01-17,NaT,NaT,NaT,187.0,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11634,STDR696,OD,2023-06-13,0,NaT,NaT,NaT,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11635,STDR696,OS,2021-06-29,0,2021-12-16,2022-06-14,2023-06-13,NaT,170.0,0.0,...,1.0,NaN,1.0,NaN,NaN,0.0,NaN,NaN,NaN,0.0
11636,STDR696,OS,2021-12-16,0,2022-06-14,2023-06-13,NaT,NaT,180.0,1.0,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0
11637,STDR696,OS,2022-06-14,1,2023-06-13,NaT,NaT,NaT,364.0,1.0,...,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [90]:
for i in range(1, 5):
    mask = (df_shift_date_label[f"date_diff_{i}"] >= (730 - 30)) & (
        df_shift_date_label[f"date_diff_{i}"] <= (730 + 30)
    )
    df_shift_date_label[f"label_730_{i}"] = df_shift_date_label[f"label_up_shift_{i}"][
        mask
    ]


df_shift_date_label["label_730"] = (
    df_shift_date_label[["label_730_1", "label_730_2", "label_730_3", "label_730_4"]]
    .bfill(axis=1)
    .iloc[:, 0]
)

In [91]:
df_shift_date_label

,pid,eye,date,label,date_up_shift_1,date_up_shift_2,date_up_shift_3,date_up_shift_4,date_diff_1,label_up_shift_1,...,label_183_1,label_183_2,label_183_3,label_183_4,label_183,label_730_1,label_730_2,label_730_3,label_730_4,label_730
0,STDR001,OD,2015-07-28,1,2016-07-14,2017-01-17,NaT,NaT,352.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,STDR001,OD,2016-07-14,2,2017-01-17,NaT,NaT,NaT,187.0,2.0,...,2.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN
5,STDR001,OD,2017-01-17,2,NaT,NaT,NaT,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,STDR001,OS,2015-07-28,0,2016-07-14,2017-01-17,NaT,NaT,352.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,STDR001,OS,2016-07-14,0,2017-01-17,NaT,NaT,NaT,187.0,0.0,...,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11634,STDR696,OD,2023-06-13,0,NaT,NaT,NaT,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11635,STDR696,OS,2021-06-29,0,2021-12-16,2022-06-14,2023-06-13,NaT,170.0,0.0,...,0.0,NaN,NaN,NaN,0.0,NaN,NaN,1.0,NaN,1.0
11636,STDR696,OS,2021-12-16,0,2022-06-14,2023-06-13,NaT,NaT,180.0,1.0,...,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
11637,STDR696,OS,2022-06-14,1,2023-06-13,NaT,NaT,NaT,364.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [92]:
print(
    sum(df_shift_date_label.label_183.notna()),
    sum(df_shift_date_label.label_365.notna()),
    sum(df_shift_date_label.label_730.notna()),
)

4206 4629 2934


In [95]:
df_shift_date_label_clean = df_shift_date_label[
    ["label", "pid", "eye", "date", "label_183", "label_365", "label_730"]
]

In [97]:
cols = ["183", "365", "730"]
for col in cols:
    df_shift_date_label_clean[f"label_{col}_diff"] = (
        df_shift_date_label_clean[f"label_{col}"] - df_shift_date_label_clean["label"]
    )

df_shift_date_label_clean

/tmp/ipykernel_9037/1326538723.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_shift_date_label_clean[f"label_{col}_diff"] = (
/tmp/ipykernel_9037/1326538723.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_shift_date_label_clean[f"label_{col}_diff"] = (
/tmp/ipykernel_9037/1326538723.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydat

,label,pid,eye,date,label_183,label_365,label_730,label_183_diff,label_365_diff,label_730_diff
0,1,STDR001,OD,2015-07-28,NaN,2.0,NaN,NaN,1.0,NaN
3,2,STDR001,OD,2016-07-14,2.0,NaN,NaN,0.0,NaN,NaN
5,2,STDR001,OD,2017-01-17,NaN,NaN,NaN,NaN,NaN,NaN
8,0,STDR001,OS,2015-07-28,NaN,0.0,NaN,NaN,0.0,NaN
10,0,STDR001,OS,2016-07-14,0.0,NaN,NaN,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
11634,0,STDR696,OD,2023-06-13,NaN,NaN,NaN,NaN,NaN,NaN
11635,0,STDR696,OS,2021-06-29,0.0,1.0,1.0,0.0,1.0,1.0
11636,0,STDR696,OS,2021-12-16,1.0,NaN,NaN,1.0,NaN,NaN
11637,1,STDR696,OS,2022-06-14,NaN,1.0,NaN,NaN,0.0,NaN


In [101]:
# 检查有多少label_diff小于0
print(
    sum(df_shift_date_label_clean.label_183_diff < 0),
    sum(df_shift_date_label_clean.label_365_diff < 0),
    sum(df_shift_date_label_clean.label_730_diff < 0),
)

205 302 260


In [102]:
# 检查有多少label_diff大于1
print(
    sum(df_shift_date_label_clean.label_183_diff > 1),
    sum(df_shift_date_label_clean.label_365_diff > 1),
    sum(df_shift_date_label_clean.label_730_diff > 1),
)

39 58 67


In [103]:
# 将label_diff小于0变为0, 将label_diff大于1变为1
cols = ["183", "365", "730"]
for col in cols:
    df_shift_date_label_clean[f"label_{col}_diff_binary"] = df_shift_date_label_clean[
        f"label_{col}_diff"
    ].copy()

    mask = df_shift_date_label_clean[f"label_{col}_diff_binary"] < 0
    df_shift_date_label_clean[f"label_{col}_diff_binary"][mask] = 0

    mask = df_shift_date_label_clean[f"label_{col}_diff_binary"] > 1
    df_shift_date_label_clean[f"label_{col}_diff_binary"][mask] = 1

df_shift_date_label_clean

/tmp/ipykernel_9037/3649698314.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_shift_date_label_clean[f"label_{col}_diff_binary"] = df_shift_date_label_clean[
/tmp/ipykernel_9037/3649698314.py:9: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = v

,label,pid,eye,date,label_183,label_365,label_730,label_183_diff,label_365_diff,label_730_diff,label_183_diff_binary,label_365_diff_binary,label_730_diff_binary
0,1,STDR001,OD,2015-07-28,NaN,2.0,NaN,NaN,1.0,NaN,NaN,1.0,NaN
3,2,STDR001,OD,2016-07-14,2.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN
5,2,STDR001,OD,2017-01-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0,STDR001,OS,2015-07-28,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN
10,0,STDR001,OS,2016-07-14,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11634,0,STDR696,OD,2023-06-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11635,0,STDR696,OS,2021-06-29,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0
11636,0,STDR696,OS,2021-12-16,1.0,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN
11637,1,STDR696,OS,2022-06-14,NaN,1.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN


In [104]:
df_shift_date_label_clean.to_excel("df_shift_date_label_clean.xlsx")

In [110]:
df_shift_date_label_clean

,label,pid,eye,date,label_183,label_365,label_730,label_183_diff,label_365_diff,label_730_diff,label_183_diff_binary,label_365_diff_binary,label_730_diff_binary
0,1,STDR001,OD,2015-07-28,NaN,2.0,NaN,NaN,1.0,NaN,NaN,1.0,NaN
3,2,STDR001,OD,2016-07-14,2.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN
5,2,STDR001,OD,2017-01-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0,STDR001,OS,2015-07-28,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN
10,0,STDR001,OS,2016-07-14,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11634,0,STDR696,OD,2023-06-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11635,0,STDR696,OS,2021-06-29,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0
11636,0,STDR696,OS,2021-12-16,1.0,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN
11637,1,STDR696,OS,2022-06-14,NaN,1.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN


In [116]:
df_shift_date_label_clean = df_shift_date_label_clean.rename(
    columns={"label": "label_fix_inconsit"}
)

In [120]:
df["date"] = pd.to_datetime(df["date"])

df_risk_level = df.merge(
    df_shift_date_label_clean, on=["pid", "eye", "date"], how="left"
)

df_risk_level

,label,macula_filename,pid,eye,date,time,label_fix_inconsit,label_183,label_365,label_730,label_183_diff,label_365_diff,label_730_diff,label_183_diff_binary,label_365_diff_binary,label_730_diff_binary
0,1,STDR001_OD_20150728_105647_Color_001.tif,STDR001,OD,2015-07-28,10:56:47,1,NaN,2.0,NaN,NaN,1.0,NaN,NaN,1.0,NaN
1,2,STDR001_OD_20160714_101437_Color_001.tif,STDR001,OD,2016-07-14,10:14:37,2,2.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN
2,2,STDR001_OD_20160714_101504_Color_001.tif,STDR001,OD,2016-07-14,10:15:04,2,2.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN
3,2,STDR001_OD_20160714_101606_Color_001.tif,STDR001,OD,2016-07-14,10:16:06,2,2.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN
4,2,STDR001_OD_20170117_110223_Color_001.tif,STDR001,OD,2017-01-17,11:02:23,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11364,0,STDR696_OD_20230613_145815_Color_001.tif,STDR696,OD,2023-06-13,14:58:15,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11365,0,STDR696_OS_20210629_160514_Color_001.tif,STDR696,OS,2021-06-29,16:05:14,0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0
11366,0,STDR696_OS_20211216_152742_Color_001.tif,STDR696,OS,2021-12-16,15:27:42,0,1.0,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN
11367,1,STDR696_OS_20220614_160840_Color_001.tif,STDR696,OS,2022-06-14,16:08:40,1,NaN,1.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN


In [124]:
print(
    sum(df_risk_level.label_183.notna()),
    sum(df_risk_level.label_365.notna()),
    sum(df_risk_level.label_730.notna()),
)

5906 6228 4000


In [122]:
df_risk_level.to_excel("macula_table_risk_level.xlsx", index=False)